This notebook reproduces a [visualization by the Wall Street Journal](https://graphics.wsj.com/infectious-diseases-and-vaccines/#b02g20t20w15) about the incidence of measles over time, which the brilliant [Brian Granger](https://github.com/ellisonbg) adapted into an [example for the Altair library](https://nbviewer.jupyter.org/github/ellisonbg/altair/blob/master/altair/notebooks/12-Measles.ipynb).

Most examples work across multiple plotting backends, this example is also available for:

* [Matplotlib measles_example](../matplotlib/measles_example.ipynb)

In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv
from holoviews import opts

hv.extension('bokeh')

## Declaring data

In [ ]:
url = 'https://raw.githubusercontent.com/blmoore/blogR/master/data/measles_incidence.csv'
data = pd.read_csv(url, skiprows=2, na_values='-')

yearly_data = data.drop('WEEK', axis=1).groupby('YEAR').sum().reset_index()
measles = pd.melt(yearly_data, id_vars=['YEAR'], var_name='State', value_name='Incidence')

heatmap = hv.HeatMap(measles, label='Measles Incidence')
aggregate = hv.Dataset(heatmap).aggregate('YEAR', np.mean, np.std)

vline = hv.VLine(1963)
marker = hv.Text(1964, 800, 'Vaccine introduction', halign='left')

agg = hv.ErrorBars(aggregate) * hv.Curve(aggregate)

## Plot

In [ ]:
overlay = (heatmap + agg * vline * marker).cols(1)
overlay.opts(
    opts.HeatMap(width=900, height=500, tools=['hover'], logz=True,
                 invert_yaxis=True, labelled=[], toolbar='above',
                 xaxis=None, colorbar=True, clim=(1, np.nan)),
    opts.VLine(line_color='black'),
    opts.Overlay(width=900, height=200, show_title=False, xrotation=90))